In [7]:
import os

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB0

In [8]:
base_dir = r"D:\user's\work related\to deepsight ai"
learning_rate = 0.001
epochs = 20
epoch_steps = 50

In [9]:
train_dir = os.path.join(base_dir, 'Training')
validation_dir = os.path.join(base_dir, 'Validation')
test_dir = os.path.join(base_dir, 'Testing')

In [10]:
train_datagenerator = ImageDataGenerator(rescale=1. / 255., rotation_range=40, width_shift_range=0.2,
                                         height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
validation_datagenerator = ImageDataGenerator(rescale=1. / 255.)
train_generator = train_datagenerator.flow_from_directory(train_dir, batch_size=20, class_mode='categorical', color_mode='rgb', shuffle=True,
                                                          target_size=(256, 256))
validation_generator = validation_datagenerator.flow_from_directory(validation_dir, batch_size=20, class_mode='categorical', color_mode='rgb', shuffle=True,
                                                                    target_size=(256, 256))

test_datagenerator = ImageDataGenerator(rescale=1./255.)
test_generator = test_datagenerator.flow_from_directory(test_dir, target_size=(256, 256), batch_size=1,class_mode=None, shuffle=False)

Found 1440 images belonging to 6 classes.
Found 480 images belonging to 6 classes.
Found 480 images belonging to 1 classes.


In [ ]:
base_vgg16model = VGG16(input_shape=(256, 256, 3), include_top=False, weights='imagenet', pooling='max')

for layer in base_vgg16model.layers:
    layer.trainable = False

output_layer = layers.Flatten()(base_vgg16model.output)
output_layer = layers.Dense(1024, activation='relu')(output_layer)
output_layer = layers.Dropout(0.2)(output_layer)
output_layer = layers.Dense(6, activation='softmax')(output_layer)

vgg16_model = tf.keras.models.Model(base_vgg16model.input, output_layer)
vgg16_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate), metrics=['acc'],
                    loss=tf.keras.losses.categorical_crossentropy)

vgghist = vgg16_model.fit(train_generator, validation_data=validation_generator, steps_per_epoch=epoch_steps, epochs=epochs)

Epoch 1/20
50/50 [==============================] - 141s 3s/step - loss: 2.3560 - acc: 0.2280 - val_loss: 1.9850 - val_acc: 0.2896
Epoch 2/20
50/50 [==============================] - 133s 3s/step - loss: 1.7542 - acc: 0.3250 - val_loss: 1.5631 - val_acc: 0.3896
Epoch 3/20
50/50 [==============================] - 132s 3s/step - loss: 1.6505 - acc: 0.3360 - val_loss: 1.4874 - val_acc: 0.4354
Epoch 4/20
50/50 [==============================] - 134s 3s/step - loss: 1.5459 - acc: 0.3820 - val_loss: 1.4643 - val_acc: 0.4167
Epoch 5/20
50/50 [==============================] - 132s 3s/step - loss: 1.4787 - acc: 0.4270 - val_loss: 1.4173 - val_acc: 0.4396
Epoch 6/20
50/50 [==============================] - 133s 3s/step - loss: 1.4683 - acc: 0.4150 - val_loss: 1.3529 - val_acc: 0.4729
Epoch 7/20
50/50 [==============================] - 133s 3s/step - loss: 1.4645 - acc: 0.4190 - val_loss: 1.3453 - val_acc: 0.4812
Epoch 8/20
50/50 [==============================] - 133s 3s/step - loss: 1.3504 - a

In [ ]:
base_inceptionmodel = InceptionV3(input_shape = (256, 256, 3), include_top=False, weights='imagenet', pooling='max')

for layer in base_inceptionmodel.layers:
    layer.trainable = False

output_layer = layers.Flatten()(base_inceptionmodel.output)
output_layer = layers.Dense(1024, activation='relu')(output_layer)
output_layer = layers.Dropout(0.2)(output_layer)
output_layer = layers.Dense(6, activation='softmax')(output_layer)

inception_model = tf.keras.models.Model(base_inceptionmodel.input, output_layer)
inception_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate), metrics=['acc'],
                    loss=tf.keras.losses.categorical_crossentropy)

inceptionhist = inception_model.fit(train_generator, validation_data=validation_generator, steps_per_epoch=epoch_steps, epochs=epochs)

In [ ]:
base_resnetmodel = ResNet50(input_shape=(256, 256,3), include_top=False, weights="imagenet", pooling='max')

for layer in base_resnetmodel.layers:
    layer.trainable = False
    
resnet_model = Sequential()
resnet_model.add(base_resnetmodel)
resnet_model.add(layers.Dense(6, activation='softmax'))
resnet_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate), metrics=['acc'],
                    loss=tf.keras.losses.categorical_crossentropy)

resnethist = resnet_model.fit(train_generator, validation_data=validation_generator, steps_per_epoch=epoch_steps, epochs=epochs)

In [ ]:
base_efficientmodel = EfficientNetB0(input_shape=(256, 256, 3), include_top=False, weights='imagenet', pooling='max')

for layer in base_efficientmodel.layers:
    layer.trainable = False
    
output_layer = base_efficientmodel.output
output_layer = layers.Flatten()(output_layer)
output_layer = layers.Dense(1024, activation="relu")(output_layer)
output_layer = layers.Dropout(0.2)(output_layer)
output_layer = layers.Dense(6, activation="softmax")(output_layer)

efficient_model = tf.keras.models.Model(base_efficientmodel.input, output_layer)
efficient_model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=learning_rate), metrics=['acc'],
                    loss=tf.keras.losses.categorical_crossentropy)

efficienthist = efficient_model.fit(train_generator, validation_data=validation_generator, steps_per_epoch=epoch_steps, epochs=epochs)

In [ ]:
pred = inception_model.predict(test_generator)